# Church and Scott Encoded Lists

An operation that handles the cons and nil cases of a list deconstruction completely defines a list. This is the Scott encoding:

In [88]:
:ext RankNTypes
:m Control.Monad

newtype ListS a = ListS { runListS :: forall r. (a -> ListS a -> r) -> r -> r }

instance Functor ListS where
  fmap f (ListS xs) = ListS $ \cons nil -> xs (\a as -> cons (f a) (fmap f as)) nil
  
instance Applicative ListS where
  pure a = ListS $ \cons _ -> cons a mempty
  l1@(ListS f) <*> l2@(ListS a) = ListS $ \cons nil ->
    f (\x xs -> a (\y ys -> cons (x y) $ (x <$> ys) <> (xs <*> l2)) nil
      )
      nil
    
instance Monad ListS where
  (ListS ma) >>= amb = ListS $ \cons nil ->
    ma (\x xs -> runListS (amb x <> (xs >>= amb)) cons nil
       )
       nil

instance Foldable ListS where
  foldr arr r (ListS xs) = xs go r where
    go a xs' = arr a $ foldr arr r xs'
    
instance Semigroup (ListS a) where
  (ListS a) <> l2@(ListS b) = ListS $ \cons nil ->
    a (\x xs -> cons x $ xs <> l2) $ b cons nil
    
instance Monoid (ListS a) where
  mempty = ListS $ \_ nil -> nil
  
instance Traversable ListS where
  traverse f (ListS xs) = xs go $ pure mempty
    where
      go a xs' = consS <$> f a <*> traverse f xs'

consS :: a -> ListS a -> ListS a
consS x xs = ListS $ \cons _ -> cons x xs

unconsS :: ListS a -> (a -> ListS a -> r) -> r -> r
unconsS = runListS

fromListS :: [a] -> ListS a
fromListS xs = ListS $ \cons nil ->
  case xs of
    (x : xs') -> cons x $ fromListS xs'
    []        -> nil

We can also define a list solely in terms of a fold operation. This is the Church list:

In [84]:
newtype ListC a = ListC { runListC :: forall r. (a -> r -> r) -> r -> r }

instance Functor ListC where
  fmap f (ListC xs) = ListC $ \arr r -> xs (arr . f) r
  
instance Applicative ListC where
  pure a = ListC $ \arr r -> arr a r
  (ListC f) <*> (ListC a) = ListC $ \brr r ->
    f (\fn r' -> a (\x r'' -> brr (fn x) r'') r') r
    
instance Monad ListC where
  (ListC ma) >>= amb = ListC $ \brr r ->
    ma (\a r' -> runListC (amb a) brr r') r
    
instance Foldable ListC where
 foldr arr r xs = runListC xs arr r
 
instance Semigroup (ListC a) where
  (ListC a) <> (ListC b) = ListC $ \arr r ->
    a arr (b arr r)
    
instance Monoid (ListC a) where
  mempty = ListC $ \_ r -> r

instance Traversable ListC where
  traverse f (ListC xs) = xs go $ pure mempty
    where
      go a r = consC <$> f a <*> r
  
consC :: a -> ListC a -> ListC a
consC a (ListC xs) = ListC $ \arr r -> arr a $ xs arr r

unconsC :: (a -> ListC a -> r) -> r -> ListC a -> r
unconsC cons nil (ListC xs) =
  case xs go (Nothing, mempty) of
    (Just a, xs') -> cons a xs'
    (Nothing, _) -> nil
  where
    go a (Just a', xs') = (Just a, consC a' xs')
    go a (Nothing, xs') = (Just a, xs')

fromListC :: [a] -> ListC a
fromListC xs = ListC $ \arr r -> foldr arr r xs

Lets try it out

In [86]:

listc :: ListC Int
listc = foldMap pure [1,2,3,4,5]

toList $ (*) <$> listc <*> listc

toList $ do
  x <- fromListC [1,2,3]
  y <- fromListC [4,5]
  pure $ x ^ y

lists :: ListS Int
lists = foldMap pure [1,2,3,4,5]

toList $ (*) <$> lists <*> lists
toList $ do
  x <- fromListS [1,2,3] :: ListS Int
  y <- fromListS [4,5]
  pure $ x ^ y


[1,2,3,4,5,2,4,6,8,10,3,6,9,12,15,4,8,12,16,20,5,10,15,20,25]

[1,1,16,32,81,243]

[1,2,3,4,5,2,4,6,8,10,3,6,9,12,15,4,8,12,16,20,5,10,15,20,25]

[1,1,16,32,81,243]